In [1]:
import os
os.chdir('..')

In [12]:
# To help with reading and manipulating data
import pandas as pd
import numpy as np

# To help with data visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# To be used for missing value imputation
from sklearn.impute import SimpleImputer

# To help with model building
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

# To get different metric scores, and split data
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score,
    ConfusionMatrixDisplay,
)

# To oversample and undersample data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# To be used for data scaling and one hot encoding
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# To be used for tuning the model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# To use statistical functions
import scipy.stats as stats

# To be used for creating pipelines and personalizing them
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To supress warnings
import warnings

warnings.filterwarnings("ignore")


#File reader
from src.ingest_data import DataIngestorFactory
from src.missing_value_imputation import *
from src.data_splitter import *
from src.model_selection import *
from src.tune_model import ModelTunning  # assuming saved as model_tunning.py

In [3]:
file_path = '/Users/hanli/cost-minimization_ML/data/Training_raw/Train.csv'
file_extension = os.path.splitext(file_path)[1]
data_ingestor = DataIngestorFactory.get_data_ingestor(file_extension)
df = data_ingestor.ingest(file_path)  # This will work now

In [4]:
missing_value_handler = MissingValueHandler(DropMissingValue(axis=0, thresh=3))
missing_value_handler.set_strategy(FillMissingValue(method='median'))
df_filled = missing_value_handler.handle_missing_values(df)

2025-06-09 01:00:09,906 - INFO - Switching missing value handling strategy.
2025-06-09 01:00:09,907 - INFO - Executing missing value handling strategy.
2025-06-09 01:00:09,908 - INFO - Filling missing value with median strategy
2025-06-09 01:00:09,948 - INFO - Missing values filled.


In [5]:
data_splitter = DataSplitter(SimpleTrainTestSplitStrategy(test_size=0.2))
X_train, X_test, y_train, y_test = data_splitter.split(df_filled, target_column='Target')

2025-06-09 01:00:11,051 - INFO - Splitting data using the selected strategy.
2025-06-09 01:00:11,052 - INFO - Performing simple train-test split.
2025-06-09 01:00:11,069 - INFO - Train-test split completed.


In [6]:
models = []  # Empty list to store all the models

# Appending models into the list

models.append(
    ("Logistic Regression", LogisticRegression(solver="newton-cg", random_state=1))
)
models.append(("dtree", DecisionTreeClassifier(random_state=1)))
models.append(("Xgboost", XGBClassifier(random_state=1, eval_metric="logloss")))

In [7]:
strategy = CrossValidationEvaluator(n_split=5, random_state=42)
evaluator = ModelEvaluator(strategy)

# Evaluate models
results = evaluator.evaluate_models(models, X_train, y_train)

# Get the best model
best_model_name, best_model = evaluator.get_best_model(models)

# Save the best model (correct static method call)
model_path = ModelEvaluator.save_best_model(best_model_name, best_model)


2025-06-09 01:00:15,328 - INFO - Evaluating models with maintenance cost optimization
2025-06-09 01:00:15,329 - INFO - Starting CV evaluation with maintenance cost scoring
2025-06-09 01:00:16,139 - INFO - Logistic Regression: Mean cost ratio = 1.4217 (±0.0090)
2025-06-09 01:00:29,087 - INFO - dtree: Mean cost ratio = 1.7411 (±0.0214)
2025-06-09 01:00:30,812 - INFO - Xgboost: Mean cost ratio = 2.0721 (±0.0401)
2025-06-09 01:00:30,813 - INFO - Best model 'Xgboost' saved to: models/Xgboost_base.pkl


In [ ]:
print('test')

In [13]:
tuner = ModelTunning(config_dir="config")

In [17]:
# Perform hyperparameter tuning
tuned_model = tuner.tune(best_model_name, best_model, X_train, y_train, cv=5, n_iter=30, random_state=42)

# Save the tuned model
model_path = tuner.save_tuned_model(best_model_name, tuned_model)
print(f"Tuned model saved at: {model_path}")

2025-06-09 01:03:24,465 - INFO - Loaded tuning config for Xgboost, from config/Xgboost.yaml
2025-06-09 01:03:24,465 - INFO - Starting hyperparameter tuning for Xgboost ...
Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x14a461a60>>
Traceback (most recent call last):
  File "/Users/hanli/cost-minimization_ML/.venv/lib/python3.12/site-packages/xgboost/core.py", line 585, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument

KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <bound method DataIter._next_wrapper of <xgboost.data.SingleBatchInternalIter object at 0x127bb3050>>
Traceback (most recent call last):
  File "/Users/hanli/cost-minimization_ML/.venv/lib/python3.12/site-packages/xgboost/core.py", line 585, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument

KeyboardInte

KeyboardInterrupt: 